# Training of Neural Network for Novozyme Prediction
This NB allows you to test a NN model and get training infos
The different parts are:
- Simple train: train a NN based on the parameters in the config file
- Compute learning curve: mse for a dataset of 100/1000/3000/all rows
- Compute feature importance: use saved models and scalers to compute the avg mse with each features being randomized one at a time
- Compute submission: use saved models and scalers to compute the avg mse for each row of the submission dataset

Note: for hyper parameters optimization see the wandb_training.py script, in which we use wandb to sweep through a list of posssible hyper parameters

In [1]:
import copy
import torch.nn.functional as F
import torch.nn as nn
import torch
import numpy as np
import pandas as pd
import time

from training_utils.file_utils import (open_json, write_json, save_submission,
                                       log_kfold_training, log_learning_curve)
from training_utils.models import SimpleNN
from training_utils.model_utils import *
from training_utils.training import train_model, k_fold_training


In [2]:
config = open_json("simple_nn_config.json")
features_dict = open_json(f"{config['dataset_dir']}/features.json")
features, features_infos = compute_feature_list(config, features_dict)

LOG_NAME = "SimpleNN"
KEEP_MODELS = True

SIMPLE_TRAIN = True
COMPUTE_LEARNING_CURVE = False
COMPUTE_FEATURE_IMPORTANCE = False
COMPUTE_SUBMISSION = True


In [3]:
if torch.cuda.is_available() and config["use_cuda"]:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)


cpu


## Simple train


In [4]:
if SIMPLE_TRAIN:
    df = load_dataset(config, features)

    all_training_results = {"simple_train": [],
                            "total_training_time": 0}
    print(f"{type(all_training_results)=}")

    # add protein_index to the dataset and get ksplit:
    df = split_dataset(df, config)
    training_results, model_list, scaler_list = k_fold_training(
        df, config, features, features_infos, device, keep_models=KEEP_MODELS)

    # add training results to all the other ones
    all_training_results["simple_train"] = training_results
    print(f"{type(all_training_results)=}")
    # compute avg_mse and time
    train_mse = sum(x["train_mse"]
                    for x in training_results)/config["kfold"]
    test_mse = sum(x["test_mse"]
                   for x in training_results)/config["kfold"]
    training_time = sum(x["time"] for x in training_results)

    # save results to output
    model = SimpleNN(
        len(features_infos["direct_features"]), config)
    model_structure = str(model).replace('(', '').replace(')', '').split('\n')
    dir_path = log_kfold_training(
        LOG_NAME, all_training_results, config, features, model_structure)
        
    if KEEP_MODELS:
        save_models_and_scalers(dir_path, model_list, scaler_list)
        
    print(f"{type(all_training_results)=}")
    print(f"logged training in {dir_path}")


## Find worst samples


In [5]:
if SIMPLE_TRAIN:
    WORST_SAMPLES_DIFF_THRESHOLD = 10
    worst_of_the_worst = []
    # WORST_SAMPLES_DIFF_THRESHOLD = 1
    for res in all_training_results.get("simple_train"):
        worst_samples = res.get("worst_samples")
        worst_of_the_worst += [record for record in worst_samples if record["diff"]
                               > WORST_SAMPLES_DIFF_THRESHOLD]

    print(len(worst_of_the_worst))
    write_json("worst_samples.json", worst_of_the_worst)


## Learning Curve


In [6]:
if COMPUTE_LEARNING_CURVE:
    df = load_dataset(config, features)

    # plot the learning curve of the model
    # ie. the avg mse when df has 10, 100, 1000 elements
    # len(df) = 5k
    num_rows = [100, 1000, 3000, len(df)]
    all_training_results = {"training_by_num_rows": [],
                            "learning_curve": {"num_rows": num_rows,
                                               "train_mse": [],
                                               "test_mse": []
                                               },
                            "total_training_time": 0
                            }

    t0 = time.time()
    for n in num_rows:
        print(f"training on {n} rows from the dataset")
        df_n_rows = df.sample(n)
        # add protein_index to the dataset and get ksplit:
        df_n_rows, ksplit = split_dataset(df_n_rows, config)
        training_results, _, _ = k_fold_training(
            df_n_rows, ksplit, config, features, features_infos, device)

        # add training results to all the other ones
        all_training_results["training_by_num_rows"].append(training_results)
        # compute avg_mse and time
        train_mse = sum(x["train_mse"]
                        for x in training_results)/config["kfold"]
        test_mse = sum(x["test_mse"]
                       for x in training_results)/config["kfold"]
        training_time = sum(x["time"] for x in training_results)

        # update result variables
        all_training_results["total_training_time"] += training_time
        all_training_results["learning_curve"]["train_mse"].append(train_mse)
        all_training_results["learning_curve"]["test_mse"].append(test_mse)

    total_time = time.time()-t0
    print(f"total_training_time= {all_training_results['total_training_time']:.2f}, {total_time= :.2f}, \
        training_time: {(all_training_results['total_training_time']/total_time)*100:.2f}% of total time")


In [7]:
if COMPUTE_LEARNING_CURVE:
    # save results to output
    model = SimpleNN(
        len(features_infos["direct_features"]), config)
    model_structure = str(model).replace('(', '').replace(')', '').split('\n')
    dir_path = log_learning_curve(
        LOG_NAME, all_training_results, config, features, model_structure)
    print(f"logged training in {dir_path}")


## Feature Importance


In [8]:
if COMPUTE_FEATURE_IMPORTANCE:
    # all_training_results[feature1]: result obtained when df[feature1] = random
    all_training_results = {}

    df = load_dataset(config, features)

    # 1. we train kfold models using all features:
    df = split_dataset(df, config)
    training_results, model_list, scaler_list = k_fold_training(
        df, config, features, features_infos, device, keep_models=True)

    # compute mse (with no feature randomized)
    none_random_train_mse = sum(x['train_mse']
                                for x in training_results)/config['kfold']
    none_random_test_mse = sum(x['test_mse']
                               for x in training_results)/config['kfold']
    all_training_results["none_randomized"] = {
        "train_mse": none_random_train_mse, "test_mse": none_random_test_mse}

    # 2. for each feature we take the df, randomize the feature column, then compute the new mse score with each model
    print("now testing with random features")
    for feature in features:
        # create a copy of the dataset with the feature column being random
        df_random = copy.deepcopy(df)
        df_random[feature] = np.random.uniform(
            low=df[feature].min(), high=df[feature].max(), size=df_random.shape[0])
        training_results = []

        # evaluate all model on this new dataset
        for k in range(config["kfold"]):
            # we get the same ksplit as during training as we already added the kfold column to df
            train = list(range(config["kfold"]))
            test = [train.pop(k)]
            df_train = df_random[df_random["kfold"].isin(train)]
            df_test = df_random[df_random["kfold"].isin(test)]
            model = model_list[k]  # model result from the training
            dataset_train_scaler = scaler_list[k]  # scaler from training

            # we load the data for evaluation
            X_train, y_train = prepare_eval_data(
                df_train, config, features, features_infos, dataset_train_scaler)
            X_test, y_test = prepare_eval_data(
                df_test, config, features, features_infos, dataset_train_scaler)

            # Evaluate this model:
            model.eval()
            with torch.set_grad_enabled(False):
                train_mse, _ = evaluate_model(X_train, y_train, model, device)
                test_mse, _ = evaluate_model(X_test, y_test, model, device)
                # print(f"MSE obtained for kfold {k}: {mse}")
                results = {
                    "train_mse": train_mse,
                    "test_mse": test_mse
                }
                training_results.append(results)

        # compute avg_mse
        train_mse = sum(x['train_mse']
                        for x in training_results)/config['kfold']
        test_mse = sum(x['test_mse']
                       for x in training_results)/config['kfold']
        # add training results to all the other ones
        all_training_results[feature] = {
            "train_mse": train_mse, "test_mse": test_mse,
            "delta_train_mse": train_mse-none_random_train_mse,
            "delta_test_mse": test_mse-none_random_test_mse,
        }

    # rank results
    ranked_delta_test_mse = [[x, all_training_results[x].get(
        "delta_test_mse", 0)] for x in all_training_results.keys()]
    ranked_delta_test_mse.sort(key=lambda x: x[1], reverse=True)
    ranked_delta_test_mse = {x[0]: x[1] for x in ranked_delta_test_mse}
    write_json(f"./outputs/{LOG_NAME}_feature_importance.json",
               {"ranked_delta_test_mse": ranked_delta_test_mse,
                "all_training_results": all_training_results})


# Predicting on submission


In [9]:
if COMPUTE_SUBMISSION:
    dir_path = "outputs/SimpleNN_11/"
    df_test = pd.read_csv(
        f"{config['dataset_dir']}/submission_all_features_filled_nan.csv")
    results = []

    model_list, scaler_list = load_models_and_scalers(dir_path)

    for k in range(len(model_list)):
        X_test = df_test[features].copy()

        model = model_list[k]  # model result from the training
        dataset_train_scaler = scaler_list[k]  # scaler from training

        # Evaluate this model:
        model.eval()
        with torch.set_grad_enabled(False):
            X_test = dataset_train_scaler.transform(X_test)
            X_test = X_test[:, features_infos["direct_features"]]

            X_test = torch.from_numpy(X_test)
            X_test = X_test.to(device)
            y_test = model(X_test)
            results.append(y_test.detach().numpy())

    submission = pd.DataFrame(columns=["seq_id", "tm"])
    submission["seq_id"] = df_test["seq_id"]
    submission["tm"] = np.mean(np.array(results), axis=0)
    print(submission.head())
    save_path = save_submission(submission, LOG_NAME)
    print(f"{save_path=}")


   seq_id        tm
0   31390  0.032039
1   31391 -0.036185
2   31392 -0.989702
3   31393 -0.371179
4   31394 -0.264186
save_path='./submissions/SimpleNN_2022-11-25_15-33-56.csv'
